In [1]:
!pip3 install --upgrade spacy
!python3 -m spacy download en_core_web_lg
!python3 -m spacy validate

!pip3 install neuralcoref

Requirement already up-to-date: spacy in c:\users\skamenshchikov\anaconda3\lib\site-packages (2.3.2)


"python3" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"python3" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
import spacy
print('REQUIERED spacy:',spacy.__version__)
from spacy import displacy

import string
import re

from bs4 import BeautifulSoup
import requests

import neuralcoref

import en_core_web_sm

nlp = en_core_web_sm.load()
neuralcoref.add_to_pipe(nlp)

REQUIERED spacy: 2.3.2


C:\Users\skamenshchikov\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\skamenshchikov\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\skamenshchikov\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
C:\Users\skamenshchikov\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible mo

In [3]:
htmlString = requests.get('https://medium.com/@sergeykamenshchikov/nlp-for-composition-of-value-tree-d8495534ebd5')
soup = BeautifulSoup(htmlString.content, 'html.parser')
paragraphs = soup.find_all(['p', 'article', 'section', 'title', 'h1', 'h2', 'h3'])
rawtext = ' '.join([i.text for i in paragraphs])

In [4]:
#check potential number of clauses, if it is more or equals N
def check_min_num_of_clauses(spacy_sentence, n):
    result=[] 
    for token in spacy_sentence:
        if (token.dep_ in ['nsubj','nsubjpass','csubj']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX'):
            result.append(token.head.text)
    if len(result)>=n:
        return True
       
    else:
        return False
#get indexes of full sentences without duplicates
def get_sents_ids_whitelist(spacy_sents):
    whitelist=[]
    i=1
    sents_texts=[]
    for sent in spacy_sents:
        if (sent.text not in sents_texts) and check_min_num_of_clauses(sent,1):
            whitelist.append(i)
            sents_texts.append(sent.text)
        i=i+1
    return(whitelist)

#cut chosen subtrees
def compress(spacy_sents,sents_whitelist):
    blacklist_tokens=[]
    
    n=1
    for sent in spacy_sents:
        if (n in sents_whitelist):
            for token in sent:
                if token.dep_ in ['appos','advmod']:
                    token_sub_tree=token.subtree
                    for t in token_sub_tree:
                        blacklist_tokens.append(t.i)
            
        n=n+1
    return(blacklist_tokens)

#get tokens for full sentences from remaining subtrees
def get_list_valid_tokens(spacy_sents,sents_whitelist,blacklist_tokens):
    valid_tokens=[]
    n=1
    for sent in spacy_sents:
        if (n in sents_whitelist):
            for token in sent:
                if (token.i not in blacklist_tokens):
                    valid_tokens.append(token.text)
                    
        n=n+1
    return(valid_tokens)

#build text from tokens
def detokenizer(list_of_tokens):
    text_str="".join([" "+w if not w.startswith("'") and not w.startswith("’") and w!='' and w not in string.punctuation else w for w in list_of_tokens]).strip()
    return(text_str)


#get all subj-verb centeres and add their subtrees
def solve_complex_sents(spacy_sents):
    out_sentences=[]
    for sent in spacy_sents:
        if check_min_num_of_clauses(sent,2):
            min_sent=[]
            head_clause=True
            for token in sent:
                if (token.dep_ in ['nsubj','nsubjpass','csubj']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX') and head_clause:
                    s=token.head.subtree
                    for t in s:
                        min_sent.append(t.text)
                    out_sentences.append(detokenizer(min_sent))
                    min_sent=[]
                if (token.dep_ in ['nsubj','nsubjpass','csubj']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX') and not head_clause:
                    s=token.head.subtree
                    for t in s:
                        if t.dep_!='mark':
                            min_sent.append(t.text)
                    out_sentences.append(detokenizer(min_sent))
                    min_sent=[]
                head_clause=False
        else:
            out_sentences.append(sent.text)
    
    return(out_sentences)

#fix sentences after cutting subtrees and extracting subtrees
def sentence_grammar_fix(sentences):
    fixed=[]
    for sent in sentences:
        sent=sent+'.'
        sent=re.sub(r'([,.\-—:])+',r'\1',sent)
        if sent[0] in [',','-','—']:
            sent=sent[1:]
        sent=sent.strip()
        if sent[0].isalpha():
            sent=sent[0].upper()+sent[1:]
        fixed.append(sent)
    return(fixed)

#processor
def spacy_processor(rawtext):
    doc = nlp(rawtext)
    sents_whitelist=get_sents_ids_whitelist(doc.sents)
    tokens_blacklist=compress(doc.sents,sents_whitelist)
    valid_tokens=get_list_valid_tokens(doc.sents,sents_whitelist,tokens_blacklist)
    text=detokenizer(valid_tokens)
    
    doc = nlp1(text)
    text_resolved=doc._.coref_resolved
    
    doc=nlp(text_resolved)
    final_sentences_list=solve_complex_sents(doc.sents)
    
    final_sentences_list=sentence_grammar_fix(final_sentences_list)
    return(' '.join(final_sentences_list))

In [ ]:
%%time
final_text=spacy_processor(rawtext)
print(final_text)